In [13]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


데이터셋 수 조절 및 압축 해제

In [ ]:
import pandas as pd
import zipfile
import os

# 엑셀 파일 경로
xlsx_path = '/content/drive/MyDrive/감정 데이터셋 조절.xlsx'

# 압축 파일 경로
zip_file_path = '/content/drive/MyDrive/음성/5차년도_2차.zip'

# 압축을 푼 파일을 저장할 폴더 경로
extract_path = '/content/drive/MyDrive/데이터셋'

# 엑셀 파일 읽기
df = pd.read_excel(xlsx_path)

# wav_id 목록 가져오기
wav_ids = df['wav_id'].tolist()

# 압축 파일 해제
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # zip 파일 내의 파일 목록 가져오기
    zip_file_list = zip_ref.namelist()

    # wav_id 목록에 해당하는 파일만 압축 해제
    for file_name in zip_file_list:
        # 파일 이름에서 wav_id 추출
        current_wav_id = os.path.splitext(file_name)[0]

        # wav_id 목록에 있는 경우에만 압축 해제
        if current_wav_id in wav_ids:
            # 파일 압축 해제
            extracted_file_path = os.path.join(extract_path, os.path.basename(file_name))
            with open(extracted_file_path, 'wb') as extracted_file:
                extracted_file.write(zip_ref.read(file_name))

print("모든 압축 파일을 처리했습니다.")

sklearn 오류 처리 위한 코드

In [ ]:
# pip ungrade 먼저 하고 런타임 다시 시작해야 함
!pip install --upgrade --ignore-installed pip setuptools

In [ ]:
# 다시 시작 후 이거 실행
!pip install -U scikit-learn --no-cache-dir

In [ ]:
# 그러면 sklearn 제대로 깔림
!pip install sklearn



---



In [ ]:
#필요한 모듈 설치하기
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch
#!pip install sklearn
!pip install pandas
!pip install numpy
!pip install kobert-transformers
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install speechrecognition

In [10]:
# 기본 라이브러리 불러오기
import pandas as pd
import numpy as np
import os
import sys

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

import librosa
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt

# 경고메세지 숨기기
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import gluonnlp as nlp
from tqdm import tqdm, tqdm_notebook
import pandas as pd
import numpy as np

#kobert라이브러리에서 많이 쓰이는 함수 불러오기
from kobert_transformers.utils import get_tokenizer
from kobert_transformers import get_kobert_model, get_distilkobert_model
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

#transformers에서 하이퍼파라미터 세팅
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
import pickle

import speech_recognition as sr

In [11]:
#GPU 사용(권장)
device = torch.device("cuda:0")
#CPU 사용
#device = torch.device("cpu")

In [15]:
#감성대화정보 엑셀로 불러오기
chatbot_data = pd.read_excel('/content/drive/MyDrive/감정 데이터셋 조절.xlsx')

# 데이터 개수
len(chatbot_data)

#데이터 구조 살펴보기
chatbot_data.sample(n=3)

,wav_id,발화문,감정_대분류
3663,5f85bd29111dfd48d40fdf7e,너는 안 당해봐서 몰라. 다른 직원들한테는 친절하게 하는데 나한테만 자꾸 뭐라고 하...,분노
4861,5f8864d8d338b948c4e69ac2,지금 당장 짜장면에서 벌레가 나왔다고 항의 전화를 해봐야겠어.,당황
5016,5fb4933c576e9378b67abf52,해피가 막 뛰어가는데 다행히 다친 사람은 없었어.,당황


In [16]:
# 감정데이터 정수형(수치)로 바꿔주기 (정수 인코딩)
chatbot_data.loc[(chatbot_data['감정_대분류'] == "불안"), 'Emotion'] = 0  #공포 => 0
chatbot_data.loc[(chatbot_data['감정_대분류'] == "상처"), 'Emotion'] = 1  #놀람 => 1
chatbot_data.loc[(chatbot_data['감정_대분류'] == "분노"), 'Emotion'] = 2  #분노 => 2
chatbot_data.loc[(chatbot_data['감정_대분류'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
chatbot_data.loc[(chatbot_data['감정_대분류'] == "당황"), 'Emotion'] = 4  #중립 => 4
chatbot_data.loc[(chatbot_data['감정_대분류'] == "기쁨"), 'Emotion'] = 5  #행복 => 5

In [17]:
# 음성데이터 라벨 및 경로데이터만 뽑아오기
data_path = chatbot_data.loc[:,['감정_대분류','wav_id', '발화문']]
data_path.columns = ['Emotions','Path', 'Text']

# 음성데이터 경로수정해주기(압축푼 폴더로 경로수정)
data_path['Path'] = '/content/drive/MyDrive/데이터셋/'+data_path['Path']+'.wav'

data_path.tail(5)

,Emotions,Path,Text
6495,불안,/content/drive/MyDrive/데이터셋/5f8ff3d0111df...,"아, 이거 어떡하지? 119 불러야 되나?"
6496,불안,/content/drive/MyDrive/데이터셋/5fbe26d2cb503...,나 엘리베이터에 갇혔어!
6497,불안,/content/drive/MyDrive/데이터셋/5fbb65a1576e9...,1층에서 4층 올라갈려고 엘리베이터 탔는데 갇혔어.
6498,불안,/content/drive/MyDrive/데이터셋/5f62b60b2e23c...,집안까지 물은 들어오지 않아서 나는 괜찮아.
6499,불안,/content/drive/MyDrive/데이터셋/5fba373d576e9...,어. 나 완전 무서워.


In [18]:
# 음성데이터를 waveplot, spectrogram 이미지로 표현

def create_waveplot(data, sr, e):
    plt.figure(figsize=(10, 3))
    plt.title('Waveplot for audio with {} emotion'.format(e), size=15)
    librosa.display.waveplot(data, sr=sr)
    plt.show()

def create_spectrogram(data, sr, e):
    # stft 함수는 음성을 시간기반에서 주파수기반으로 변환함
    X = librosa.stft(data)
    Xdb = librosa.amplitude_to_db(abs(X))
    plt.figure(figsize=(12, 3))
    plt.title('Spectrogram for audio with {} emotion'.format(e), size=15)
    librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')
    plt.colorbar()

In [19]:
# 음성데이터 argumentation을 통해서 데이터 수 늘리기 및 오버피팅 방지
def noise(data):
    noise_amp = 0.035*np.random.uniform()*np.amax(data)
    data = data + noise_amp*np.random.normal(size=data.shape[0])
    return data

def stretch(data, rate=0.8):
    return librosa.effects.time_stretch(data, rate)

def shift(data):
    shift_range = int(np.random.uniform(low=-5, high = 5)*1000)
    return np.roll(data, shift_range)

def pitch(data, sampling_rate, pitch_factor=0.7):
    return librosa.effects.pitch_shift(data, sampling_rate, pitch_factor)

# 예시
path = np.array(data_path.Path)[1]
data, sample_rate = librosa.load(path)

# 음성의 특성추출하는 함수 ( MFCC, MEL, RMSV)

def extract_features(data):

    result = np.array([])

    # MFCC
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mfcc)) # stacking horizontally

    # Root Mean Square Value
#    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
#    result = np.hstack((result, rms)) # stacking horizontally

    # MelSpectogram
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mel)) # stacking horizontally

    return result

# 음성의 특성을 추출한 데이터를 축적하는 함수 (Argumentation된 데이터도 같이)
def get_features(path):
    # duration과 offset은 각 오디오 파일의 시작과 끝에서 오디오가 없는 것을 처리
    data, sample_rate = librosa.load(path, duration=2.5, offset=0.6)

    # 원래데이터
    res1 = extract_features(data)
    result = np.array(res1)
    # 노이즈가 추가된 데이터
    noise_data = noise(data)
    res2 = extract_features(noise_data)
    result = np.vstack((result, res2)) # 병렬적으로 추가

    # 피칭및 스트레칭된 데이터
    # new_data = stretch(data)
    # data_stretch_pitch = pitch(new_data, sample_rate)
    # res3 = extract_features(data_stretch_pitch)
    # result = np.vstack((result, res3)) # 병렬적으로 추가

    return result

In [ ]:
 # 이미 실행해서 features.csv에 저장했으므로 이 코드/아래 코드 실행 안 해도 됨
 X, Y, Z = [], [], []
 for path, emotion, text in zip(data_path.Path, data_path.Emotions, data_path.Text):
     feature = get_features(path)
     for ele in feature:
         X.append(ele)
         # 2개의 augmentation이 존재하므로 총 한개의 음성데이터가 3개의 음성데이터가 됨
         Y.append(emotion)
         Z.append(text)

In [12]:
# 위에서 추출한 Features를 csv 파일로 저장
# Features = pd.DataFrame(X)
# Features['labels'] = Y
# Features['text'] = Z
# Features.to_csv('/content/drive/MyDrive/음성/features.csv', index=False)

In [21]:
# features를 불러옴
Features = pd.read_csv('/content/drive/MyDrive/음성/features.csv')

In [22]:
X = Features.iloc[: ,:-2].values  # Audio
Y = Features['labels'].values
Z = Features['text'].values  # Text

In [23]:
le = LabelEncoder()
le = le.fit(Y)
Y = le.transform(Y)
with open('/content/drive/MyDrive/음성/label_encoder.pkl','wb') as f:
  pickle.dump(le, f)

In [24]:
# 훈련데이터, 테스트데이터 분리 및 개수 확인
x_train, x_test, y_train, y_test = train_test_split(X, Y, random_state=0)
z_train, z_test, _, _ = train_test_split(Z, Y, random_state=0)
x_train.shape, y_train.shape, x_test.shape, y_test.shape, z_train.shape, z_test.shape

((9750, 148), (9750,), (3250, 148), (3250,), (9750,), (3250,))

In [25]:
# 음성 데이터 스케일 조정
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
x_train.shape, y_train.shape, x_test.shape, y_test.shape
with open('/content/drive/MyDrive/음성/scaler.pkl','wb') as f:
  pickle.dump(scaler, f)

In [26]:
# 위에서 저장한 내용 확인, StandardScaler()각 출력되면 됨 (그 안에 객체 저장됨)

import pickle

# scaler.pkl 파일을 열어서 읽기 모드로 불러오기
with open('/content/drive/MyDrive/음성/scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

# scaler 객체의 내용 확인
print(scaler)


StandardScaler()


In [27]:
# 음성 데이터의 차원 모델에 맞게 통일
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((9750, 148, 1), (9750,), (3250, 148, 1), (3250,))

In [28]:
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)
x_test = torch.tensor(x_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.int64)

In [29]:
audio_train_ds = TensorDataset(x_train, y_train)
audio_test_ds = TensorDataset(x_test, y_test)
audio_train_dataloader = torch.utils.data.DataLoader(audio_train_ds, batch_size=64)
audio_test_dataloader = torch.utils.data.DataLoader(audio_test_ds, batch_size=64)

In [30]:
# 텍스트 데이터 BERT 모델에 들어가기 위한 dataset을 만들어주는 클래스
class BERTDataset(Dataset):
    def __init__(self, data, label, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i]) for i in data]
        self.labels = [np.int32(i) for i in label]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [32]:
# 이미 학습된 kobert 모델 (komodel_1119.tar) pkl 파일로 변환

import torch
import pickle

# .tar 파일에서 모델 및 관련 정보 불러오기
checkpoint_path = "/content/drive/MyDrive/음성/komodel_1119_all.tar"
checkpoint = torch.load(checkpoint_path, map_location=torch.device("cpu"))

# 상태 사전 추출
model_state_dict = checkpoint['model']

# 다른 필요한 정보 추출 (예: vocab, config 등)
# 필요한 정보가 있다면 해당 부분을 추가

# kobert_data.pkl 파일에 저장
kobert_data = {
    'model_state_dict': model_state_dict,
    # 다른 필요한 정보 추가
}

with open('/content/drive/MyDrive/음성/kobert_data.pkl', 'wb') as f:
    pickle.dump(kobert_data, f)

In [ ]:
# pkl 정상적으로 변환됐는지 내용 확인

import pickle

# 저장된 pkl 파일 열기
with open('/content/drive/MyDrive/음성/kobert_data.pkl', 'rb') as f:
    kobert_data_loaded = pickle.load(f)

# 내용 확인
print(kobert_data_loaded)

In [50]:
with open('/content/drive/MyDrive/음성/kobert_data.pkl', 'wb') as f:
  # BERT 모델, Vocabulary 불러오기
  tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
  bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
  vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
  pickle.dump([bertmodel, vocab, tokenizer],f)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [52]:
with open('/content/drive/MyDrive/음성/kobert_data.pkl', 'rb') as f:
    bertmodel, vocab, _= pickle.load(f)

# 필요한 값 추출 (아래 코드 주석 지우지 X)
#bertmodel = data['model_state_dict']
#vocab = data['vocab']

In [53]:
# #문장을 토크나이저를 통해서 토큰으로 변환(토큰화)
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

In [55]:
# 데이터셋 BERT에 맞게 적용
tok = tokenizer.tokenize

max_len = 50
text_train_ds = BERTDataset(z_train, y_train, tok, vocab, max_len, True, False)
text_test_ds = BERTDataset(z_test, y_test, tok, vocab, max_len, True, False)
#bert모델에 넣기위해 torch형식으로 변환
text_train_dataloader = torch.utils.data.DataLoader(text_train_ds, batch_size=64, num_workers=5)
text_test_dataloader = torch.utils.data.DataLoader(text_test_ds, batch_size=64, num_workers=5)

모델 설계

In [56]:
# 파라미터 세팅 ( 문장길이, 모델배치수,에폭수 ...)
max_len = 50
batch_size = 64
warmup_ratio = 0.1
num_epochs = 10
learning_rate =  3e-5

1) 음성 모델

In [57]:
# 음성 모델 만들기

class AudioClassifier(nn.Module):
    def __init__(self):
        super(AudioClassifier, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=x_train.shape[1], out_channels=256, kernel_size=1, stride=1,padding='same')
        self.max_pool1 = nn.MaxPool1d(5, stride=2, padding = 2)

        self.conv2 = nn.Conv1d(in_channels=256, out_channels=256, kernel_size=5, stride=1, padding='same')
        self.max_pool2 = nn.MaxPool1d(5, stride=2, padding = 2)

        self.conv3 = nn.Conv1d(in_channels=256, out_channels=128, kernel_size=5, stride=1, padding='same')
        self.max_pool3 = nn.MaxPool1d(5, stride=2, padding = 2)
        self.dropout1 = nn.Dropout(0.2)

        self.conv4 = nn.Conv1d(in_channels=128, out_channels=64, kernel_size=5, stride=1, padding='same')
        self.max_pool4 = nn.MaxPool1d(5, stride=2, padding = 2)

        self.fc1 = nn.Linear(64, 32)
        self.dropout2 = nn.Dropout(0.6)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.max_pool1(x)

        x = F.relu(self.conv2(x))
        x = self.max_pool2(x)

        x = F.relu(self.conv3(x))
        x = self.max_pool3(x)
        x = self.dropout1(x)

        x = F.relu(self.conv4(x))
        x = self.max_pool4(x)

        x = torch.flatten(x,1)

        x = F.relu(self.fc1(x))
        x = self.dropout2(x)

        return x

In [ ]:
import torch
print(torch.cuda.is_available())

In [59]:
model1 = AudioClassifier().to(device)

2) 텍스트 모델

In [60]:
# KoBERT 학습모델 만들기
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=32,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [61]:
#BERT 모델 불러오기
model2 = BERTClassifier(bertmodel,  dr_rate=0.6).to(device)

3) 멀티모달 모델

In [62]:
# 멀티모달 모델 만들기

class MultimodalClassifier(nn.Module):
    def __init__(self, audioModel, textModel):
        super(MultimodalClassifier, self).__init__()
        self.audioModel = audioModel
        self.textModel = textModel

        self.fc1 = nn.Linear(64,32)
        self.dropout1 = nn.Dropout(0.2)

        self.fc2 = nn.Linear(32, 5)

    def forward(self, x1, token_ids, valid_length, segment_ids):
        x1 = self.audioModel(x1)
        x2 = self.textModel(token_ids, valid_length, segment_ids)
        x = torch.cat((x1,x2), 1)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)

        x = F.relu(self.fc2(x))
        return x

In [80]:
import torch

# 위에서 정의한 MultimodalClassifier 모델 객체를 생성
# (audioModel, textModel은 각각 음성 모델과 텍스트 모델을 나타내는 것으로 가정)
multimodal_model = MultimodalClassifier(model1, model2)

# 모델을 저장할 경로와 파일 이름
save_path = '/content/drive/MyDrive/음성/multimodal.pt'

# 모델의 state_dict를 저장
torch.save(multimodal_model.state_dict(), save_path)

# 저장된 state_dict 로드
state_dict = torch.load(save_path)

# MultimodalClassifier 모델 생성
model3 = MultimodalClassifier(model1, model2).to(device) # .to(device)로 GPU에서 동작함

# 로드한 state_dict 적용
model3.load_state_dict(state_dict)

criterion = nn.CrossEntropyLoss()

# optimizer 정의
optimizer = optim.AdamW(model3.parameters(), lr=0.0001)

In [78]:
model3 = MultimodalClassifier(model1, model2).to(device)
model3 = torch.load('/content/drive/MyDrive/음성/multimodal.pt')

In [81]:
for epoch in range(num_epochs):
  cost, total, correct = 0,0,0
  for (i, (X, Y)), (_, (token_ids, valid_length, segment_ids, _)) in zip(enumerate(audio_train_dataloader), enumerate(text_train_dataloader)):
    optimizer.zero_grad()
    X, token_ids, valid_length, segment_ids, Y = X.to(device), token_ids.to(device), valid_length.to(device), segment_ids.to(device), Y.to(device)
    outputs = model3(X,token_ids, valid_length, segment_ids)
    loss = criterion(outputs, Y)
    loss.backward()

    optimizer.step()

    cost += loss
    if i % 10 == 9:
      print(f'epoch: {epoch} | batch_id: {i} | loss: {loss.item()}') # 밑에 test로 수정
  for (i, (X, Y)), (_, (token_ids, valid_length, segment_ids, _)) in zip(enumerate(audio_test_dataloader), enumerate(text_test_dataloader)):
    X, token_ids, valid_length, segment_ids, Y = X.to(device), token_ids.to(device), valid_length.to(device), segment_ids.to(device), Y.to(device)
    outputs = model3(X,token_ids, valid_length, segment_ids)
    total += Y.size(0)
    correct += sum(torch.argmax(outputs, 1) == Y)
  avg_cost = cost / len(audio_train_dataloader)
  accuracy = 100*correct/total

  print(f'\nepoch: {epoch} | loss: {avg_cost}\naccuracy: {accuracy}\n')

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


epoch: 0 | batch_id: 9 | loss: 1.5914589166641235
epoch: 0 | batch_id: 19 | loss: 1.5600335597991943
epoch: 0 | batch_id: 29 | loss: 1.4503648281097412
epoch: 0 | batch_id: 39 | loss: 1.4168198108673096
epoch: 0 | batch_id: 49 | loss: 1.2999228239059448
epoch: 0 | batch_id: 59 | loss: 1.2356876134872437
epoch: 0 | batch_id: 69 | loss: 1.5438776016235352
epoch: 0 | batch_id: 79 | loss: 1.047884464263916
epoch: 0 | batch_id: 89 | loss: 1.0391483306884766
epoch: 0 | batch_id: 99 | loss: 1.1911214590072632
epoch: 0 | batch_id: 109 | loss: 0.9127700924873352
epoch: 0 | batch_id: 119 | loss: 1.000382900238037
epoch: 0 | batch_id: 129 | loss: 0.8680084943771362
epoch: 0 | batch_id: 139 | loss: 0.8116167783737183
epoch: 0 | batch_id: 149 | loss: 1.0599135160446167

epoch: 0 | loss: 1.1973462104797363
accuracy: 65.63076782226562

epoch: 1 | batch_id: 9 | loss: 0.7863821387290955
epoch: 1 | batch_id: 19 | loss: 0.7983085513114929
epoch: 1 | batch_id: 29 | loss: 0.7929604649543762
epoch: 1 | batc

In [82]:
y_pred = np.array([])
y_test = np.array([])
for (i, (X, Y)), (_, (token_ids, valid_length, segment_ids, _)) in zip(enumerate(audio_test_dataloader), enumerate(text_test_dataloader)):
  X, token_ids, valid_length, segment_ids, Y = X.to(device), token_ids.to(device), valid_length.to(device), segment_ids.to(device), Y.to(device)
  outputs = model3(X,token_ids, valid_length, segment_ids)
  y_pred = np.append(y_pred, np.array(torch.argmax(outputs,1).cpu()))
  y_test = np.append(y_test, np.array(Y.cpu()))

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [83]:
print(classification_report(y_test, y_pred))  # class 한글로 뜨게

              precision    recall  f1-score   support

         0.0       0.85      0.94      0.90       627
         1.0       0.87      0.94      0.90       707
         2.0       0.97      0.87      0.92       639
         3.0       0.96      0.92      0.94       630
         4.0       0.95      0.90      0.93       647

    accuracy                           0.92      3250
   macro avg       0.92      0.92      0.92      3250
weighted avg       0.92      0.92      0.92      3250



실제 데이터 모델 성능 테스트 (아직 미완성)

In [84]:
cd /content/drive/MyDrive/완성

/content/drive/MyDrive/완성


In [88]:
# 대화 정보 엑셀로 불러오기
data = pd.read_excel('/content/drive/MyDrive/감정 데이터셋 조절.xlsx')

print(data['감정_대분류'].value_counts())

기쁨    1300
슬픔    1300
분노    1300
당황    1300
불안    1300
Name: 감정_대분류, dtype: int64


In [99]:
# 음성데이터 라벨 및 경로데이터만 뽑아오기
#감정 및 음성데이터경로만 뽑아오기
data_path = data.loc[:,['감정_대분류','wav_id']]
data_path.columns = ['Emotions','Path']

# 음성데이터 경로수정해주기(압축푼 폴더로 경로수정)
data_path['Path'] = '/content/drive/MyDrive/데이터셋/'+data_path['Path']+'.wav'

data_path.tail(5)

,Emotions,Path
6495,불안,/content/drive/MyDrive/데이터셋/5f8ff3d0111df...
6496,불안,/content/drive/MyDrive/데이터셋/5fbe26d2cb503...
6497,불안,/content/drive/MyDrive/데이터셋/5fbb65a1576e9...
6498,불안,/content/drive/MyDrive/데이터셋/5f62b60b2e23c...
6499,불안,/content/drive/MyDrive/데이터셋/5fba373d576e9...


In [100]:
# 감정데이터 정수형(수치)로 바꿔주기 (정수 인코딩)
data_path[(data_path['Emotions'] == "불안"), 'Emotion'] = 0  #공포 => 0
data_path[(data_path['Emotions'] == "상처"), 'Emotion'] = 1  #놀람 => 1
data_path[(data_path['Emotions'] == "분노"), 'Emotion'] = 2  #분노 => 2
data_path[(data_path['Emotions'] == "슬픔"), 'Emotion'] = 3  #슬픔 => 3
data_path[(data_path['Emotions'] == "당황"), 'Emotion'] = 4  #중립 => 4
data_path[(data_path['Emotions'] == "기쁨"), 'Emotion'] = 5  #행복 => 5

TypeError: ignored

In [101]:
data_path = data_path.sample(frac=1).reset_index(drop=True)  # shuffling하고 index reset
data_path

,Emotions,Path
0,불안,/content/drive/MyDrive/데이터셋/5fb9ce254c55e...
1,슬픔,/content/drive/MyDrive/데이터셋/5fb9f753576e9...
2,불안,/content/drive/MyDrive/데이터셋/5f9805c69e04b...
3,기쁨,/content/drive/MyDrive/데이터셋/5f5cc0442e23c...
4,분노,/content/drive/MyDrive/데이터셋/5fb33fb948cdd...
...,...,...
6495,슬픔,/content/drive/MyDrive/데이터셋/5f871b27f8fac...
6496,불안,/content/drive/MyDrive/데이터셋/5f3f4f389dd51...
6497,분노,/content/drive/MyDrive/데이터셋/5f427bc79dd51...
6498,당황,/content/drive/MyDrive/데이터셋/5fb8733a44697...


In [102]:
target = data_path['Path'].iloc[1]
target

'/content/drive/MyDrive/데이터셋/5fb9f753576e9378b67ac6b7.wav'

In [103]:
r = sr.Recognizer()
harvard = sr.AudioFile(target)
with harvard as source:
  audio = r.record(source)
text = r.recognize_google(audio, language='ko-KR')

In [104]:
text

'이럴 땐 뭘 해야 좋을까'

In [105]:
# 음성의 특성추출하는 함수 ( MFCC, MEL, RMSV)
def extract_features(data):

    result = np.array([])

    # MFCC
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mfcc)) # stacking horizontally

    # Root Mean Square Value
#    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
#    result = np.hstack((result, rms)) # stacking horizontally

    # MelSpectogram
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mel)) # stacking horizontally

    return result

In [106]:
data, sample_rate = librosa.load(target, duration=2.5, offset=0.6)

In [107]:
feature = np.array(extract_features(data))

In [108]:
x = feature.reshape(1,-1)
z = text

In [109]:
# # 음성 데이터 스케일 조정
with open('/content/drive/MyDrive/음성/scaler.pkl','rb') as f:
  scaler = pickle.load(f)
x = scaler.transform(x)

In [110]:
# 음성 데이터의 차원 모델에 맞게 통일
x = np.expand_dims(x, axis=2)
x.shape

(1, 148, 1)

In [116]:
X = torch.tensor(x, dtype=torch.float32)

#문장을 토크나이저를 통해서 토큰으로 변환(토큰화)
with open('/content/drive/MyDrive/음성/kobert_data.pkl', 'rb') as f:
  bertmodel, vocab, _ = pickle.load(f)
  tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
  tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [119]:
tok = tokenizer.tokenize

transform = nlp.data.BERTSentenceTransform(
            tok, max_seq_length=50, vocab=vocab, pad=True, pair=False)

Z = list(transform(z))
Z1, Z2, Z3 = torch.tensor(Z[0].reshape(1,-1)),torch.tensor(Z[1].reshape(1,-1)),torch.tensor(Z[2].reshape(1,-1))


X.shape, Z1.shape,Z2.shape,Z3.shape

(torch.Size([1, 148, 1]),
 torch.Size([1, 50]),
 torch.Size([1, 1]),
 torch.Size([1, 50]))

In [120]:
# 음성 모델 만들기

class AudioClassifier(nn.Module):
    def __init__(self):
        super(AudioClassifier, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=x_train.shape[1], out_channels=256, kernel_size=1, stride=1,padding='same')
        self.max_pool1 = nn.MaxPool1d(5, stride=2, padding = 2)

        self.conv2 = nn.Conv1d(in_channels=256, out_channels=256, kernel_size=5, stride=1, padding='same')
        self.max_pool2 = nn.MaxPool1d(5, stride=2, padding = 2)

        self.conv3 = nn.Conv1d(in_channels=256, out_channels=128, kernel_size=5, stride=1, padding='same')
        self.max_pool3 = nn.MaxPool1d(5, stride=2, padding = 2)
        self.dropout1 = nn.Dropout(0.2)

        self.conv4 = nn.Conv1d(in_channels=128, out_channels=64, kernel_size=5, stride=1, padding='same')
        self.max_pool4 = nn.MaxPool1d(5, stride=2, padding = 2)

        self.fc1 = nn.Linear(64, 32)
        self.dropout2 = nn.Dropout(0.6)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.max_pool1(x)

        x = F.relu(self.conv2(x))
        x = self.max_pool2(x)

        x = F.relu(self.conv3(x))
        x = self.max_pool3(x)
        x = self.dropout1(x)

        x = F.relu(self.conv4(x))
        x = self.max_pool4(x)

        x = torch.flatten(x,1)

        x = F.relu(self.fc1(x))
        x = self.dropout2(x)

        return x

In [121]:
# KoBERT 학습모델 만들기
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=32,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [127]:
# 멀티모달 모델 만들기

class MultimodalClassifier(nn.Module):
    def __init__(self, audioModel, textModel):
        super(MultimodalClassifier, self).__init__()
        self.audioModel = audioModel
        self.textModel = textModel

        self.fc1 = nn.Linear(64,32)
        self.dropout1 = nn.Dropout(0.2)

        self.fc2 = nn.Linear(32, 5)

    def forward(self, x1, token_ids, valid_length, segment_ids):
        x1 = self.audioModel(x1)
        x2 = self.textModel(token_ids, valid_length, segment_ids)
        x = torch.cat((x1,x2), 1)
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)

        x = F.relu(self.fc2(x))
        return x

In [132]:
# 모델 인스턴스 생성
model = MultimodalClassifier(model1, model2)

# 저장된 state_dict 로드
model_state_dict = torch.load('/content/drive/MyDrive/음성/multimodal.pt')
model.load_state_dict(model_state_dict)

# 모델을 GPU로 이동
model = model.to(device)

In [133]:
output = model(X.to(device), Z1.to(device), Z2.to(device),Z3.to(device))

Input: 이럴 땐 뭘 해야 좋을까 / 아래 Output: 불안

In [134]:
output

tensor([[0.0000, 0.0000, 0.0000, 0.0049, 0.0000]], device='cuda:0',
       grad_fn=<ReluBackward0>)

In [135]:
le.inverse_transform([torch.argmax(output.cpu())])

array(['불안'], dtype=object)

(파이토치 pth) 모델 저장

In [138]:
# 모델 저장
torch.save(model.state_dict(), '/content/drive/MyDrive/완성/multimodal_1125.pth')

In [137]:
# 저장된 모델 불러오기
model.load_state_dict(torch.load('/content/drive/MyDrive/완성/multimodal_1125.pth'))

<All keys matched successfully>